In [6]:
import numpy as np
import random
import string

class StockMarket:
    def __init__(self, stock_tickers):
        self.stock_prices = {ticker: np.random.uniform(10, 100) for ticker in stock_tickers}
        self.deltas = {ticker: self.get_new_delta() for ticker in stock_tickers}
        self.days_since_last_change = {ticker: 0 for ticker in stock_tickers}

    def get_new_delta(self):
        return np.random.normal(0, 0.03)

    def advance_one_day(self):
        for ticker in self.stock_prices:
            if self.stock_prices[ticker] < 0.01:
                continue
            chance_of_change = 0.1 + 0.1 * self.days_since_last_change[ticker]
            if np.random.rand() < chance_of_change:
                self.deltas[ticker] = self.get_new_delta()
                self.days_since_last_change[ticker] = 0
            self.stock_prices[ticker] *= (1 + self.deltas[ticker])
            self.days_since_last_change[ticker] += 1

    def get_stock_prices(self):
        return self.stock_prices

class Portfolio:
    def __init__(self, initial_cash):
        self.cash = initial_cash
        self.stocks = {}

    def buy(self, stock_ticker, k, stock_price):
        if stock_price < 0.01:
            self.stocks[stock_ticker] = 0
            return
        total_cost = (stock_price * k) + 0.50
        if self.cash >= total_cost:
            self.cash -= total_cost
            self.stocks[stock_ticker] = self.stocks.get(stock_ticker, 0) + k

    def sell(self, stock_ticker, k, stock_price):
        if stock_price < 0.01:
            self.stocks[stock_ticker] = 0
            return
        if stock_ticker in self.stocks and self.stocks[stock_ticker] >= k:
            self.cash += (stock_price * k) - 0.50
            self.stocks[stock_ticker] -= k
            if self.stocks[stock_ticker] == 0:
                del self.stocks[stock_ticker]

def generate_stock_ticker(length=4):
    """
    randomly generates a stock ticker containing upper case letters and numbers with the given length
    """
    letters = string.ascii_uppercase
    numbers = string.digits
    ticker = ''.join(random.choice(letters + numbers) for _ in range(length))
    return ticker

STOCK_TICKERS = [generate_stock_ticker() for _ in range(6)]
INITIAL_CASH = 1000

def main():
    ## TODO: Replace the following code with your algorithm
    INITIAL_CASH=1000
    s = StockMarket(STOCK_TICKERS)
    p = Portfolio(INITIAL_CASH)
    def get_the_rate(stock_rates,ticker): 
        if stock_rates[0][ticker] == 0:
            return 0
        else:
            return (stock_rates[1][ticker]-stock_rates[0][ticker])/stock_rates[0][ticker]
    best_ticker = random.choice(STOCK_TICKERS)
    stocks = p.stocks
    for day in range(100):
        lst = []
# get the first day's price as previous price
        prev_price = s.get_stock_prices().copy()
        lst.append(prev_price)
# update the stock market
        s.advance_one_day()  
# get the second day's price, we use the first day and the second day's price to make decision.
        lst.append(s.get_stock_prices())
        rate_dict = {}
# use get rate to get the increase/decrease rate of every ticker, and choose the best one for trading.
        for ticker in STOCK_TICKERS:
            rate_dict[ticker] = get_the_rate(lst,ticker)
        max_ticker = max(rate_dict,key = rate_dict.get)
        min_ticker = min(rate_dict, key = rate_dict.get)
        if abs(rate_dict[max_ticker])<abs(rate_dict[min_ticker]):
            best_ticker = min_ticker
    # we chose 0.1 as the threshold, because the "chance_of_change" is closely related to this rate.
            if abs(rate_dict[min_ticker])*lst[0][ticker]>0.1:
                p.buy(best_ticker,1,lst[1][best_ticker])
                cash = p.cash
                INITIAL_CASH = cash
                new_stock = p.stocks
    # update the stock, get the currently best stocks.
                stocks = new_stock
        else:
            best_ticker = max_ticker
            if abs(rate_dict[min_ticker])*lst[0][ticker]>0.1:
                p.sell(best_ticker,1,lst[1][best_ticker])
                cash = p.cash
                INITIAL_CASH  = cash
                new_stock = p.stocks
                stocks = new_stock
    final_stock_value = sum(p.stocks[ticker] * s.get_stock_prices()[ticker] for ticker in p.stocks)
    final_value = p.cash + final_stock_value  
    print(f"Total value of portfolio after 100 days: ${final_value:.2f}")

if __name__ == "__main__":
    main()


Total value of portfolio after 100 days: $1184.70


## Sample Solution

*I think since `new_delta` is randomly chosen in this question. Therefore, the price change of the stock is fluctuating randomly and similar to  random walk model. Since the random walk model has no memory, so based on a long period of historical `stock_price` data to make trading decision is not reliable. Therefore this code is only based on the previous day's price fluctuations as a reference to determine whether to buy or sell in the current period.*